# McByte: Multi-Object Tracking with Mask Propagation

This notebook provides an easy-to-use interface for running McByte tracking with:
- **YOLOv8** detector (your custom weights)
- **Modular tracker selection** (McByte, ByteTrack, SORT, DeepSORT, BoT-SORT, OC-SORT)
- **SAM + Cutie** mask propagation
- **MP4 video** input/output

---

## 1. Install Dependencies

In [ ]:
# Core dependencies
!pip install -q torch torchvision --index-url https://download.pytorch.org/whl/cu118
!pip install -q ultralytics  # YOLOv8
!pip install -q segment-anything  # SAM
!pip install -q hydra-core omegaconf einops

# Tracking dependencies
!pip install -q lap filterpy cython_bbox

# Optional: Additional trackers
# !pip install -q sort-tracker  # For SORT
# !pip install -q deep-sort-realtime  # For DeepSORT
# !pip install -q ocsort  # For OC-SORT

# Clone McByte repository
!git clone https://github.com/HiteshG/McByte.git
%cd McByte

# Build C++ extensions
!python setup.py develop

## 2. Download Model Weights

In [ ]:
import os

# Create directories
os.makedirs("sam_models", exist_ok=True)
os.makedirs("mask_propagation/Cutie/weights", exist_ok=True)

# === SAM Model Selection ===
sam_type = "vit_b"  # @param ["vit_b", "vit_l", "vit_h"]

SAM_URLS = {
    "vit_b": "https://dl.fbaipublicfiles.com/segment_anything/sam_vit_b_01ec64.pth",
    "vit_l": "https://dl.fbaipublicfiles.com/segment_anything/sam_vit_l_0b3195.pth",
    "vit_h": "https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth",
}

sam_filename = f"sam_{sam_type}.pth"
sam_path = f"sam_models/{sam_filename}"

if not os.path.exists(sam_path):
    print(f"Downloading SAM {sam_type}...")
    !wget -q {SAM_URLS[sam_type]} -O {sam_path}
    print(f"SAM downloaded to: {sam_path}")
else:
    print(f"SAM already exists: {sam_path}")

# === Optional: SAM2 ===
use_sam2 = False  # @param {type:"boolean"}

if use_sam2:
    !pip install -q sam2
    from huggingface_hub import hf_hub_download
    sam2_path = hf_hub_download(
        repo_id="facebook/sam2-hiera-large",
        filename="sam2_hiera_large.pt",
        local_dir="sam_models"
    )
    sam_type = "sam2"
    sam_path = sam2_path
    print(f"SAM2 downloaded to: {sam_path}")

# === Cutie ===
cutie_path = "mask_propagation/Cutie/weights/cutie-base-mega.pth"

if not os.path.exists(cutie_path):
    print("Downloading Cutie weights...")
    !wget -q https://github.com/hkchengrex/Cutie/releases/download/v1.0/cutie-base-mega.pth \
        -O {cutie_path}
    print(f"Cutie downloaded to: {cutie_path}")
else:
    print(f"Cutie already exists: {cutie_path}")

print("\n=== Model weights ready ===")

## 3. Configuration

Configure your tracking settings here.

In [ ]:
# =============================================================================
# YOUR CONFIGURATION - MODIFY THESE VALUES
# =============================================================================

# === YOLOv8 Model ===
# Upload your .pt file to Colab and set the path here
YOLOV8_WEIGHTS = "/content/your_model.pt"  # @param {type:"string"}

# Your model's class names (in order)
CLASS_NAMES = ["person", "ball"]  # @param

# === Tracker Selection ===
# Options: "mcbyte", "bytetrack", "sort", "deepsort", "botsort", "ocsort"
TRACKER_TYPE = "mcbyte"  # @param ["mcbyte", "bytetrack", "sort", "deepsort", "botsort", "ocsort"]

# === Tracking Parameters ===
TRACK_THRESH = 0.6  # @param {type:"slider", min:0.1, max:0.9, step:0.05}
TRACK_BUFFER = 30  # @param {type:"slider", min:10, max:100, step:5}

# === SAM Configuration ===
# Uses values from previous cell
SAM_CHECKPOINT = sam_path
SAM_TYPE = sam_type

# === Cutie Configuration ===
CUTIE_WEIGHTS = cutie_path

# === Other Options ===
ENABLE_MASKS = True  # @param {type:"boolean"}
VIS_TYPE = "basic"  # @param ["basic", "no_vis"]
CONF_THRESH = 0.01  # @param {type:"number"}

print("Configuration:")
print(f"  YOLOv8 weights: {YOLOV8_WEIGHTS}")
print(f"  Class names: {CLASS_NAMES}")
print(f"  Tracker: {TRACKER_TYPE}")
print(f"  Track threshold: {TRACK_THRESH}")
print(f"  Track buffer: {TRACK_BUFFER}")
print(f"  SAM type: {SAM_TYPE}")
print(f"  Masks enabled: {ENABLE_MASKS}")

## 4. Validation Checks

Verify that all components are working correctly.

In [ ]:
import torch
import cv2
import numpy as np
import sys

def validate_setup():
    """Validate all components are working."""
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"[1/4] Device: {device} ({'GPU: ' + torch.cuda.get_device_name(0) if device == 'cuda' else 'CPU'})")

    # === YOLOv8 Check ===
    print("[2/4] Checking YOLOv8...")
    try:
        from ultralytics import YOLO
        model = YOLO(YOLOV8_WEIGHTS)
        test_img = np.random.randint(0, 255, (640, 640, 3), dtype=np.uint8)
        results = model(test_img, verbose=False)
        print(f"  [OK] YOLOv8 loaded: {len(model.names)} classes")
        print(f"  [OK] Custom classes: {CLASS_NAMES}")
        del model
    except Exception as e:
        print(f"  [FAIL] YOLOv8: {e}")
        return False

    # === SAM Check ===
    print("[3/4] Checking SAM...")
    try:
        if SAM_TYPE == "sam2":
            from sam2.build_sam import build_sam2
            from sam2.sam2_image_predictor import SAM2ImagePredictor
            sam = build_sam2(SAM_CHECKPOINT)
            sam.to(device)
            predictor = SAM2ImagePredictor(sam)
        else:
            from segment_anything import sam_model_registry, SamPredictor
            sam = sam_model_registry[SAM_TYPE](checkpoint=SAM_CHECKPOINT)
            sam.to(device)
            predictor = SamPredictor(sam)

        test_img = np.random.randint(0, 255, (640, 640, 3), dtype=np.uint8)
        predictor.set_image(test_img)
        masks, scores, _ = predictor.predict(
            box=np.array([100, 100, 300, 300]),
            multimask_output=False
        )
        print(f"  [OK] SAM working: mask shape {masks.shape}, score {scores[0]:.3f}")
        del sam, predictor
    except Exception as e:
        print(f"  [FAIL] SAM: {e}")
        return False

    # === Cutie Check ===
    print("[4/4] Checking Cutie...")
    try:
        sys.path.insert(0, "mask_propagation/Cutie")
        from cutie.model.cutie import CUTIE
        from hydra import initialize, compose
        from hydra.core.global_hydra import GlobalHydra
        from omegaconf import open_dict
        from cutie.inference.utils.args_utils import get_dataset_cfg

        GlobalHydra.instance().clear()
        initialize(version_base='1.3.2', config_path="mask_propagation/Cutie/cutie/config")
        cfg = compose(config_name="eval_config")

        with open_dict(cfg):
            cfg['weights'] = CUTIE_WEIGHTS

        _ = get_dataset_cfg(cfg)

        cutie = CUTIE(cfg).to(device).eval()
        weights = torch.load(CUTIE_WEIGHTS, map_location=device)
        cutie.load_weights(weights)
        print("  [OK] Cutie loaded successfully")
        del cutie
    except Exception as e:
        print(f"  [FAIL] Cutie: {e}")
        return False

    # Cleanup
    torch.cuda.empty_cache()

    print("\n" + "="*50)
    print("[OK] All validation checks passed!")
    print("="*50)
    return True

# Run validation
validation_passed = validate_setup()

## 5. Run Inference

Upload your video and run tracking.

In [ ]:
from google.colab import files
from tools.colab_inference import run_inference

# Upload video
print("Please upload your video file:")
uploaded = files.upload()

if uploaded:
    video_path = list(uploaded.keys())[0]
    print(f"\nUploaded: {video_path}")
else:
    raise ValueError("No video uploaded!")

In [ ]:
# Run tracking with masks
print("Starting inference...")
print("="*50)

output_path = run_inference(
    video_path=video_path,
    yolov8_weights=YOLOV8_WEIGHTS,
    tracker_type=TRACKER_TYPE,
    track_thresh=TRACK_THRESH,
    track_buffer=TRACK_BUFFER,
    sam_checkpoint=SAM_CHECKPOINT,
    sam_type=SAM_TYPE,
    cutie_weights=CUTIE_WEIGHTS,
    enable_masks=ENABLE_MASKS,
    class_names=CLASS_NAMES,
    vis_type=VIS_TYPE,
    conf_thresh=CONF_THRESH,
    verbose=True,
)

print("="*50)
print(f"\nOutput saved to: {output_path}")

## 6. Display and Download Results

In [ ]:
from IPython.display import HTML, display
from base64 import b64encode
import os

# Display video in notebook
def show_video(video_path, width=800):
    """Display video in notebook."""
    if not os.path.exists(video_path):
        print(f"Video not found: {video_path}")
        return

    # Read video file
    with open(video_path, 'rb') as f:
        mp4 = f.read()

    # Encode to base64
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

    # Display
    display(HTML(f'''
        <video width={width} controls>
            <source src="{data_url}" type="video/mp4">
            Your browser does not support the video tag.
        </video>
    '''))

print("Displaying output video...")
show_video(output_path)

In [ ]:
# Download output video
from google.colab import files

print(f"Downloading: {output_path}")
files.download(output_path)

## 7. (Optional) Process Multiple Videos

In [ ]:
# Batch processing example
import os
from tools.colab_inference import run_inference

def process_videos_batch(video_dir, output_dir="outputs"):
    """
    Process all videos in a directory.

    Args:
        video_dir: Directory containing input videos
        output_dir: Directory for output videos
    """
    os.makedirs(output_dir, exist_ok=True)

    video_extensions = ['.mp4', '.avi', '.mov', '.mkv']
    videos = [
        f for f in os.listdir(video_dir)
        if any(f.lower().endswith(ext) for ext in video_extensions)
    ]

    print(f"Found {len(videos)} videos to process")

    results = []
    for i, video_name in enumerate(videos, 1):
        print(f"\n[{i}/{len(videos)}] Processing: {video_name}")

        input_path = os.path.join(video_dir, video_name)
        output_name = f"tracked_{video_name}"
        output_path = os.path.join(output_dir, output_name)

        try:
            result = run_inference(
                video_path=input_path,
                yolov8_weights=YOLOV8_WEIGHTS,
                output_path=output_path,
                tracker_type=TRACKER_TYPE,
                track_thresh=TRACK_THRESH,
                track_buffer=TRACK_BUFFER,
                sam_checkpoint=SAM_CHECKPOINT,
                sam_type=SAM_TYPE,
                cutie_weights=CUTIE_WEIGHTS,
                enable_masks=ENABLE_MASKS,
                class_names=CLASS_NAMES,
                verbose=False,
            )
            results.append((video_name, result, "success"))
            print(f"  [OK] Output: {result}")
        except Exception as e:
            results.append((video_name, None, str(e)))
            print(f"  [FAIL] {e}")

    print("\n" + "="*50)
    print("Batch processing complete!")
    print(f"Success: {sum(1 for r in results if r[2] == 'success')}/{len(results)}")

    return results

# Example usage:
# results = process_videos_batch("/content/videos", "/content/outputs")

## 8. (Optional) Custom Tracker Configuration

In [ ]:
# Example: Using different trackers with custom parameters
from tools.tracker_interface import create_tracker, list_available_trackers

# List available trackers
print("Available trackers:")
for tracker_name in list_available_trackers():
    print(f"  - {tracker_name}")

# Example configurations for different trackers:

# McByte (default - with mask-enhanced matching)
mcbyte_tracker = create_tracker(
    tracker_type="mcbyte",
    track_thresh=0.6,
    track_buffer=30,
    cmc_method="orb",  # Camera motion compensation
)

# ByteTrack (standard, without mask enhancement)
bytetrack_tracker = create_tracker(
    tracker_type="bytetrack",
    track_thresh=0.5,
    track_buffer=30,
)

# SORT (simple Kalman + Hungarian)
sort_tracker = create_tracker(
    tracker_type="sort",
    track_buffer=30,
    min_hits=3,
    iou_threshold=0.3,
)

# DeepSORT (with ReID features)
# Note: Requires pip install deep-sort-realtime
# deepsort_tracker = create_tracker(
#     tracker_type="deepsort",
#     track_buffer=30,
#     n_init=3,
#     embedder="mobilenet",
# )

print("\nTrackers configured successfully!")

---

## Troubleshooting

### Common Issues:

1. **CUDA out of memory**: Reduce `max_internal_size` in Cutie config or use a smaller SAM model (vit_b)

2. **YOLOv8 model not found**: Make sure you've uploaded your .pt file and set the correct path

3. **SAM download failed**: Manually download from the URLs and upload to Colab

4. **Tracker package not found**: Install the required package:
   - SORT: `pip install sort-tracker`
   - DeepSORT: `pip install deep-sort-realtime`
   - OC-SORT: `pip install ocsort`

### Memory Optimization:

Edit `mask_propagation/Cutie/cutie/config/eval_config.yaml`:
```yaml
max_internal_size: 540  # Reduce from -1 (original resolution)
```

---

**McByte** - CVPRW 2025 | [GitHub](https://github.com/HiteshG/McByte)